In [41]:
include("src/readPRP.jl")
include("src/utils.jl")

detect_sous_tour (generic function with 1 method)

In [43]:
B_100 = "./PRP_instances/B_100_instance17.prp"
A_14 = "./PRP_instances/A_014_#ABS1_15_1.prp"

"./PRP_instances/A_014_#ABS1_15_1.prp"

In [ ]:
using Base.Iterators
using LinearAlgebra

In [12]:
function Bin_Packing(prp, t)
    Q = prp["Q"]
    n = prp["n"]

    somme = 0 # Poids cumulé dans la boite courante
    boites = Vector{Int64}[] # Ensemble des boites

    boiteCourante = [0] # Toutes les boites doivent contenir le centre de dépot 0

    for i in 1:n 
        if prp["Clients"].d[i][t] != 0 # On ne traite que les clients qui ont une demande pour ce pas de temps
            somme += prp["Clients"].d[i][t]

            if somme > Q
                # On enregistre la boite avant d'ajouter l'objet qui fait déborder la capacité
                push!(boites, boiteCourante)

                # On met l'objet qui a fait déborder la boite précédente dans la nouvelle boite 
                # On met donc la somme courante (du poids dans la boite courante) à jour
                somme = prp["Clients"].d[i][t]
                boiteCourante = [0]
            end
            push!(boiteCourante, i)
        end
    end
    if length(boiteCourante) > 1
        push!(boites, boiteCourante)
    end
    return boites
end

Bin_Packing (generic function with 2 methods)

In [26]:
Bin_Packing(Read_PRP_instance("./PRP_instances/A_014_#ABS1_15_1.prp"), 1)

Vector{Int64}[]

In [46]:
function binPacking(prp, t)
    
    
    cumsum = 0
    set = Vector{Int64}[]
    subset = [0]
    for i in 2:prp["n"]+1
        cumsum += prp["Clients"].d[i][t]
        if cumsum > prp["Q"]
            push!(set, subset)
            cumsum = prp["Clients"].d[i][t]
            subset = [0]
        end
        push!(subset, i)
    end
    if length(subset) > 1
        push!(set, subset)
    end
    return set
end

binPacking (generic function with 2 methods)

In [38]:
h = Read_PRP_instance("./PRP_instances/A_014_#ABS1_15_1.prp")
for t in 1:6
    println(binPacking(h, t))
end

[[0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]]
[[0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]]
[[0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]]
[[0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]]
[[0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]]
[[0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]]


In [39]:
function clark_wright(params,demands,costs,t)

    n = params["n"] #n est le nombre de clients
    Q = params["Q"] #Q est la capacité des camions


    #Initialiser S
    S = [[0, i] for i in 1:n]
    whereIs = Dict(i => circuit for (i, circuit) in enumerate(S))

    #Calcul des s_{i,j} et mettre dans l'ordre décroissant
    s=[]
    for i in 1:n, j in 1:n
        if i != j
            push!(s, [ (i,j), costs[1, i+1] + costs[1, j+1] - costs[i+1, j+1] ]) # pb indices
        end
    end
    sort!(s, by = x->x[2], rev=true) #by = key pour sort, rev = reverse (ordre décroissant)

    #Construction des circuits (attention il peut en avoir plus de m (le nombre de camion disponible))
    for k in 1:length(s)

        i=s[k][1][1]
        j=s[k][1][2]

        circuit_i = whereIs[i]
        circuit_j = whereIs[j]

        #Si i et j ne sont pas dans le même circuit
        if circuit_j != circuit_i

            index_circuit_i = findfirst(item -> item == circuit_i, S)
            index_circuit_j = findfirst(item -> item == circuit_j, S)

            circuit_union = union(S[index_circuit_i], S[index_circuit_j])

            #Calcul demande du nouveau circuit circuit_union
            demandeUnion=0
            for l in circuit_union[2:end] #commence à 2 car le premier noeud du circuit est 0 (le dépot n'a pas de demande)
                demandeUnion += demands[l][t]
            end
            
            #Si la demande du nouveau circuit n'est pas trop pour un seul camion
            if demandeUnion <= Q
                deleteat!(S, sort!([index_circuit_i, index_circuit_j]))

                push!(S, circuit_union)


                for index in circuit_union
                    push!(whereIs, index=>circuit_union)
                end

            end
        end

    end

    return S #la liste des circuits qui sont des tournées valides (qui respectent la contrainte de poids)
end

clark_wright (generic function with 1 method)

In [64]:

function cw(data, demande, t)
    """
    Paramètres : 
    data le dictionnaire contenant les données de l'instance
    demande un tableau [1:n, 1:l] de taille n*l, demande[i,t] est la quantité produite pour le revendeur i à la période t, soit q[i,t] dans la notation LSP
    t l'instant de temps à considérer
    """
    # Nous utilisons sij = c0i + c0j - αcij + β|c0i + c0j | + γ di+dj / d
    # avec α, β, γ pris entre 0 et 2 afin de prendre un peu en compte le fait
    # que la fusion dépend plus ou moins des distances et plus ou moins des valeurs des demandes
    alpha = 1
    beta = 1
    gamma = 1

    # Récupération des données
    Q = data["Q"] # Q la capacité maximale de chaque véhicule
    n = data["n"] # n le nombre de clients donc le nombre de noeuds du graphe (sans compter le dépot) ici
    cout = calcul_dist(data) # le cout de transport du client i (1:n+1) au j (1:n+1) (l'indice 1 est le centre de depot) 	
    # Pour rappel, tous les indices de cout doivent être augmentés de 1 par rapport aux indices de l'énoncé (en julia, les indices commencent à 1)

    # On créer l'ensemble de client à traiter
    clients_t = []
    d = 0
    for i in 1:n
        if demande[i+1][t] != 0 # On ne traite que les clients qui ont une demande pour ce pas de temps
            d =  d + demande[i+1][t]
            push!(clients_t, i)
        end
    end

    # Partons d’une solution à n circuits définis comme les n allers-retours de 0 à chaque client i
    solution = Array[]
    for i in clients_t 
        push!(solution, [0,i]) # Un circuit aller-retrour de 0 à i est simplement caractérisé par une boite contenant les deux sommets 0 et i
    end

    # Calcul des s[i,j] 
    s = []
    for i in clients_t 
        for j in clients_t 
            if i!=j
                # On enregistre dans s à la fois le s[i,j] mais aussi les indices (i,j) pour ne pas les perdre lors du tri
                push!(s, [(i,j), cout[1,i+1] + cout[1,j+1] - alpha*cout[i+1,j+1] + beta*abs(cout[1,i+1] + cout[1,j+1]) + gamma*(demande[i+1][t]+demande[j+1][t])/d])
            end
        end
    end
    
    # Trier s par s[i,j] décroissant
    sort!(s, by = x->x[2], rev=true) 

    # Parcours dans l'ordre des s[i,j] décroissant pour la fusion des circuits (boites)
    for k in 1:length(s)
        # On récupère les sommets i et j associés au s[i,j] que l'on traite
        i = s[k][1][1]
        j = s[k][1][2]
        boite_i = []
        boite_j = [] 
        indice_boite_i = -1
        indice_boite_j = -1

        # On cherche les circuits (boites) dans lesquels sont i et j
        for indice_boite in 1:length(solution) 
            if i in solution[indice_boite]
                boite_i = solution[indice_boite]
                indice_boite_i = indice_boite
            end

            if j in solution[indice_boite]
                boite_j = solution[indice_boite]
                indice_boite_j = indice_boite
            end

            if indice_boite_i != -1 && indice_boite_j != -1 # On a donc trouvé les deux boites que l'on cherchait
                break
            end
        end

        # Si i et j ne sont pas dans la même boite
        if indice_boite_j != indice_boite_i 
            # On crée la boite constituée des éléments de la boite de i auquel on ajoute tous les éléments de la boite de j (fusion des boites)
            boite_fusionnee = copy(boite_i) 
            for elem in boite_j
                if !(elem in boite_fusionnee)
                    push!(boite_fusionnee, elem)
                end
            end

            # On calcule la demande du nouveau circuit caractérisé par la boite boite_fusionnee
            demande_fusion = 0 
            for l in 2:length(boite_fusionnee) # Commence à 2 car les indices commencent à 1 et que le dépot est le premier élément et n'a pas de demande
                demande_fusion += demande[boite_fusionnee[l]+1][t]
            end

            # Si la réunion des deux circuits ne dépasse pas la demande transportable par un seul véhicule, réunir en un seul circuit les deux circuits
            if demande_fusion<=Q
                # On supprime les circuits associés aux boites boite_i et boite_j
                nouv_solution = []
                for indice_boite in 1:length(solution)
                    if indice_boite == indice_boite_i || indice_boite == indice_boite_j 
                        continue # On ne garde pas les anciens circuits (boites) dans lesquels étaient i ou j
                    end
                    push!(nouv_solution, solution[indice_boite])
                end
                solution = nouv_solution
                push!(solution, boite_fusionnee) # On ajoute également le circuit constitué de la fusion des deux boites
            end
        end
    end

    return solution
end

cw (generic function with 1 method)

In [65]:
h = Read_PRP_instance("./PRP_instances/A_014_#ABS1_15_1.prp")
for t in 1:6
    costs = calcul_dist(h)
    println(clark_wright(h, h["Clients"].d, costs, t))
    println(cw(h, h["Clients"].d, t))
end

[[0, 1, 7, 10, 3, 2, 5, 9, 4, 8, 13, 6, 12, 11, 14]]
Array[]
[[0, 1, 7, 10, 3, 2, 5, 9, 4, 8, 13, 6, 12, 11, 14]]
Any[[0, 1, 7, 3, 5, 2, 4, 6, 12, 8, 13, 11, 9, 10, 14]]
[[0, 1, 7, 10, 3, 2, 5, 9, 4, 8, 13, 6, 12, 11, 14]]
Any[[0, 1, 7, 3, 5, 2, 4, 6, 12, 8, 13, 11, 9, 10, 14]]
[[0, 1, 7, 10, 3, 2, 5, 9, 4, 8, 13, 6, 12, 11, 14]]
Any[[0, 1, 7, 3, 5, 2, 4, 6, 12, 8, 13, 11, 9, 10, 14]]
[[0, 1, 7, 10, 3, 2, 5, 9, 4, 8, 13, 6, 12, 11, 14]]
Any[[0, 1, 7, 3, 5, 2, 4, 6, 12, 8, 13, 11, 9, 10, 14]]
[[0, 1, 7, 10, 3, 2, 5, 9, 4, 8, 13, 6, 12, 11, 14]]
Any[[0, 1, 7, 3, 5, 2, 4, 6, 12, 8, 13, 11, 9, 10, 14]]


In [48]:
h = Read_PRP_instance(B_100)
for t in 1:6
    costs = calcul_dist(h)
    #println(clark_wright(h, h["Clients"].d, costs, t))
    println(binPacking(h, t))
    println(Bin_Packing(h, t))
end

[[0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101]]
Vector{Int64}[]
[[0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26], [0, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88], [0, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101]]
[[0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26], [0, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 